In [ ]:
!nvidia-smi

In [ ]:
import os
import random
import shutil
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
if torch.cuda.is_available():
    device = torch.device("cuda")  # GPU로 설정
    print(f"GPU 사용: {torch.cuda.get_device_name(0)}")  # GPU 이름 출력
else:
    device = torch.device("cpu")  # CPU로 설정
    print("GPU를 사용할 수 없습니다. CPU로 진행합니다.")

GPU 사용: Tesla V100-SXM2-16GB


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


## DEFS

In [ ]:
def cnt_dir(dir):

    lst = os.listdir(dir)
    cnt = len(lst)

    return cnt

In [ ]:
import os

file_name = "/content/gdrive/MyDrive/data.zip"
output_dir = "/content/"
os.system("unzip "+file_name+" -d "+output_dir)

0

In [ ]:
def move_file(source, dest, pct): # 0.3 = 30%
    files = os.listdir(source)
    num = int(len(files)*pct)

    move = random.sample(files, num)

    for file_name in move:
        source_file = os.path.join(source, file_name)
        dest_file = os.path.join(dest, file_name)
        shutil.move(source_file, dest_file)

In [ ]:
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(5, 3)),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d((2, 1)),

            nn.Conv2d(64, 128, kernel_size=(5, 3)),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d((2, 1)),

            nn.Conv2d(128, 256, kernel_size=(5, 3)),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d((2, 1)),

            nn.Conv2d(256, 512, kernel_size=(5, 3)),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d((2, 1)),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(512*7*88, 256),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.5),
            nn.Linear(256, 2)  # 마지막 레이어에서 Softmax 없이 출력
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x


## DATA PREPROCESSING

In [ ]:
train_neg_path = '/content/train/0'
train_pos_path = '/content/train/1'

val_neg_path = '/content/val/0'
val_pos_path = '/content/val/1'

# percent = 0.3

# move_file(train_neg_path, val_neg_path, percent)
# move_file(train_pos_path, val_pos_path, percent)

In [ ]:
a = cnt_dir(train_neg_path)
b = cnt_dir(train_pos_path)
c = cnt_dir(val_neg_path)
d = cnt_dir(val_pos_path)

print('train 0 폴더 파일 개수: {}\n1 폴더 파일 개수: {}'.format(a, b))
print('val 0 폴더 파일 개수: {}\n1 폴더 파일 개수: {}'.format(c, d))

train 0 폴더 파일 개수: 21543
1 폴더 파일 개수: 21543
val 0 폴더 파일 개수: 9237
1 폴더 파일 개수: 9237


In [ ]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
    transforms.Resize((180, 96)),
    transforms.RandomResizedCrop(size=(180, 96), scale=(0.8, 1.0), ratio=(0.75, 1.333)),
    transforms.GaussianBlur(kernel_size=3),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value='random')
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

val_transforms = transforms.Compose([
    transforms.Resize((180, 96)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [ ]:
train_data = datasets.ImageFolder(root='/content/train', transform=train_transforms)
val_data = datasets.ImageFolder(root='/content/val', transform=val_transforms)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
val_loader = DataLoader(val_data, batch_size=128)

## TRAIN

In [ ]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 30
model.to(device)

CNN(
  (conv_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 3), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.1)
    (2): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(5, 3), stride=(1, 1))
    (4): LeakyReLU(negative_slope=0.1)
    (5): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(5, 3), stride=(1, 1))
    (7): LeakyReLU(negative_slope=0.1)
    (8): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(256, 512, kernel_size=(5, 3), stride=(1, 1))
    (10): LeakyReLU(negative_slope=0.1)
    (11): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layers): Sequential(
    (0): Linear(in_features=315392, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.1)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in

In [ ]:
for epoch in tqdm(range(epochs)):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Epoch [{epoch + 1}/{epochs}] "
          f"Loss: {running_loss / len(train_loader):.4f} "
          f"Val Loss: {val_loss / len(val_loader):.4f} "
          f"Accuracy: {(100 * correct / total):.2f}%")

Epoch [1/30] Loss: 2.2770 Val Loss: 0.7010 Accuracy: 50.00%
Epoch [2/30] Loss: 0.6522 Val Loss: 0.6381 Accuracy: 59.43%
Epoch [3/30] Loss: 230.2478 Val Loss: 1.3166 Accuracy: 57.34%
Epoch [4/30] Loss: 0.9544 Val Loss: 0.6868 Accuracy: 57.19%
Epoch [5/30] Loss: 0.7920 Val Loss: 0.6483 Accuracy: 57.50%
Epoch [6/30] Loss: 0.7996 Val Loss: 0.6891 Accuracy: 50.00%
Epoch [7/30] Loss: 1.9326 Val Loss: 10.7591 Accuracy: 51.18%
Epoch [8/30] Loss: 1.6619 Val Loss: 0.6818 Accuracy: 50.01%
Epoch [9/30] Loss: 0.6792 Val Loss: 0.6506 Accuracy: 60.92%
Epoch [10/30] Loss: 412.5912 Val Loss: 19.9193 Accuracy: 50.00%
Epoch [11/30] Loss: 4.5257 Val Loss: 5.2304 Accuracy: 50.39%
Epoch [12/30] Loss: 1.4796 Val Loss: 0.7454 Accuracy: 57.22%
Epoch [13/30] Loss: 0.9288 Val Loss: 0.6604 Accuracy: 56.72%
Epoch [14/30] Loss: 0.9413 Val Loss: 0.6497 Accuracy: 60.73%
Epoch [15/30] Loss: 0.8080 Val Loss: 0.7164 Accuracy: 51.14%
Epoch [16/30] Loss: 0.7649 Val Loss: 0.6571 Accuracy: 57.63%
Epoch [17/30] Loss: 0.7575 

In [ ]:
# 모델 저장 경로 설정
model_path = '/content/gdrive/MyDrive/model2.pth'

# 모델 상태 저장
torch.save(model.state_dict(), model_path)

print(f"모델이 {model_path}에 저장되었습니다.")


모델이 /content/gdrive/MyDrive/model2.pth에 저장되었습니다.
